***
<a id='beginning'></a> <!--\label{beginning}-->
* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [4. The Visibility space](4_0_introduction.ipynb)
    * Previous: [4.3 The Visibility Function](4_3_The_Visibility_Function.ipynb)
    * Next: [4.4.2 UV Coverage: The N-element Interferometer](4_4_2_UV_Coverage_The_N-element_Interferometer.ipynb)

***

Main author: Julien Girard

In [3]:
from IPython.core.display import HTML
styles = open("../style/course.css", "r").read()
HTML(styles)

---
# Outline

### 4.4.1 UV Tracks

* 4.4.1.A Observing with a baseline in time
    - Variation of the projected baseline
    - Motion in the (u,v) plane
    - e.g. Earth rotation: the baseline as seen from the source
    

* 4.4.1.B Shape of the tracks
    - dependence of baseline length
    - Latitude of observation
    - General case (tracks are ellipsis)
    - parameters of the ellipsis
    
    
* 4.4.1.C Special cases
    - East-West array
    - Polar interferometer (tracks are circle)
    - Equatorial interferometer (tracks are straight lines)


---

---

---


<p class=conclusion>
  <font size=4> **Important things to remember**</font>
  <br>
  <br>
  
&bull; *The measured spatial frequency depends on the length of the **projected** baseline* <br><br>
&bull; *The measured spatial frequency depends on the length of the **projected** baseline*
</p>

### The particular case of the East-West baseline (2-elements interferometer OR Section 3 UV space)

Note: all uvtracks are contained in a plane (no need for W-projection, but just a correction between the $uv$ plane and the $XY$ plane

####  In the Equatorial $XYZ$ space

If the baseline is aligned with the East-West direction, then the Elevation $E$ of the baseline is zero and the Azimuth $A$ is $\frac{\pi}{2}$. Equation A simplifies as follow:

\begin{equation}
\frac{1}{\lambda}
\begin{bmatrix}
X\\
Y\\
Z
\end{bmatrix}
=
|\mathbf{b_\lambda}|
\begin{bmatrix}
\cos L_a \sin 0 - \sin L_a \cos 0 \cos \frac{\pi}{2}\nonumber\\ 
\cos 0 \sin \frac{\pi}{2} \nonumber\\
\sin L_a \sin 0 + \cos L_a \cos 0 \cos \frac{\pi}{2}\\
\end{bmatrix}
=
\begin{bmatrix}
0\\
|\mathbf{b_\lambda}|\\
0 \\
\end{bmatrix}
\end{equation}

This relation does not contains $H$

#### In the $uvw$ space

If we observe a source at declination $\delta_0$ but with varying Hour Angle, $H$, we will have the following relation:

\begin{equation}
\begin{pmatrix}
u\\
v\\
\end{pmatrix}
=
\begin{pmatrix}
\sin H & \cos H & 0\\ 
-\sin \delta_0 \cos H & \sin\delta_0\sin H & \cos\delta_0\\
\end{pmatrix} 
\begin{pmatrix}
0\\
|\mathbf{b_\lambda}| \\
0
\end{pmatrix}
\end{equation}

\begin{equation}
\begin{pmatrix}
u\\
v\\
\end{pmatrix}
=
\begin{pmatrix}
|\mathbf{b_\lambda}| \cos H \\ 
|\mathbf{b_\lambda}| \sin\delta_0 \sin H\\
\end{pmatrix} 
\end{equation}

when $H = 6^\text{h}$ (West)

\begin{equation}
\begin{pmatrix}
u\\
v\\
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 
|\mathbf{b_\lambda}|\sin\delta_0\\
\end{pmatrix} 
\end{equation}

when $H = 0^\text{h}$ (South)
\begin{equation}
\begin{pmatrix}
u\\
v\\
\end{pmatrix}
=
\begin{pmatrix}
|\mathbf{b_\lambda}| \\ 
0\\
\end{pmatrix} 
\end{equation}

when $H = -6^\text{h}$ (East)

\begin{equation}
\begin{pmatrix}
u\\
v\\
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 
-|\mathbf{b_\lambda}|\sin\delta_0\\
\end{pmatrix} 
\end{equation}

In this case, one case notice that we always have a relationship between $u$, $v$ and $|\mathbf{b_\lambda}|$:

$$ u^2+\left( \frac{v}{\sin\delta_0}\right) ^2=|\mathbf{b}|^2$$ 

<div class=warn>
**Warning:** the $\sin\delta_0$ factor appearing in Eq., can be interpreted as a compression factor.
</div>

We will come back to it in Section (UVspace) [EXREF]

####  From $lmn$ to ($\alpha$, $\delta$)

<div class=advice>
**Note:** The complete derivation of Eq. A from the spherical triangle **ZSSc** can be found in Appendix A.2 [INREF]
</div>

The quantity $\text{b}_\text{proj}$, namely the **projected baseline** is the module of the projection of the physical baseline *as seen* from direction $\mathbf{s}$. It can be defined as a difference of OPDs and **depends on** the choice of the phase center.

* ($x, y, z$) - Cartesian coordinates
* ($\rho,\theta,\varphi$) - Spherical coordinates
* ($l,m$, $n$) - Direction Cosines
* ($X,Y$, $Z$) - Equatorial Coordinate Reference Frame (Baselines)
* ($u,v$, $w$) - Visibility Coordinate System


* $\mathbf{\hat{e}_x},\mathbf{\hat{e}_y},\mathbf{\hat{e}_z}$ - Basis for cartesian
* $\mathbf{\hat{e}_{\rho}},\mathbf{\hat{e}_{\theta}},\mathbf{\hat{e}_{\varphi}}$ - Basis for spherical
* $\mathbf{\hat{e}_X},\mathbf{\hat{e}_Y},\mathbf{\hat{e}_Z}$ - Basis for equatorial XYZ
* $\mathbf{\hat{e}_u},\mathbf{\hat{e}_v},\mathbf{\hat{e}_w}$ - Basis for uvw
* $\mathbf{\hat{e}_l},\mathbf{\hat{e}_m},\mathbf{\hat{e}_n}$ - Basis for lmn

---
* [Go back up](#beginning) <!--\ref{beginning}--> 
* Or continue to next section: [4.4.2 UV Coverage: The N-element Interferometer -->](4_4_2_UV_Coverage_The_N-element_Interferometer.ipynb)
---